In [1]:
# Célula 1: Importar bibliotecas
import pandas as pd
import numpy as np
import os
import sys
import plotly.express as px

In [2]:

# Adiciona o diretório 'src' ao caminho do sistema para que possamos importar nosso módulo
# Isso garante que o notebook encontre o db_connector.py
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.db_connector import get_db_connection, get_raw_data

print("Bibliotecas importadas e caminho do projeto configurado com sucesso.")


Bibliotecas importadas e caminho do projeto configurado com sucesso.


In [3]:
engine = get_db_connection()

if engine:
    print("Conexão com o banco de dados estabelecida.")

    with open('../sql/Estoque REF - GRI.sql', 'r') as file:
        query_estoque = file.read()

    df_estoque_raw = get_raw_data(query_estoque, engine)


    print(f"Dados de Estoque carregados: {len(df_estoque_raw)} linhas.")
    #print(f"Dados de Fechamento carregados: {len(df_fechamento_raw)} linhas.")
else:
    print("Falha na conexão com o banco de dados.")

df_estoque = df_estoque_raw.copy()



Conexão com o banco de dados estabelecida.
Dados de Estoque carregados: 6796 linhas.


In [4]:
engine = get_db_connection()

if engine:
    print("Conexão com o banco de dados estabelecida.")
    
    
    with open('../sql/Fechamento FKM.sql', 'r', encoding='utf-8') as file:
        query_fechamento = file.read()
        
    
    
    df_fechamento_raw = get_raw_data(query_fechamento, engine)
    
    print(f"Dados do fechamento FKM carregados: {len(df_fechamento_raw)} linhas.")
    
else:
    print("Falha na conexão com o banco de dados.")
    
df_fechamento = df_fechamento_raw.copy()

Conexão com o banco de dados estabelecida.
Dados do fechamento FKM carregados: 147215 linhas.


In [7]:
# Célula de Tratamento de Datas no df_fechamento

# --- 1. Padronizar as colunas de data ---
# Primeiro, vamos garantir que a coluna 'DataCriacao' também seja do tipo datetime.
# Usar errors='coerce' é uma boa prática para evitar erros com dados inválidos.
df_fechamento['DataCriacao'] = pd.to_datetime(df_fechamento['DataCriacao'], errors='coerce')
df_fechamento['DataEntrada'] = pd.to_datetime(df_fechamento['DataEntrada'], errors='coerce')

# Verificar se alguma data falhou na conversão (opcional, mas bom para depuração)
datas_invalidas_criacao = df_fechamento['DataCriacao'].isnull().sum()
datas_invalidas_entrada = df_fechamento['DataEntrada'].isnull().sum()

if datas_invalidas_criacao > 0:
    print(f"Aviso: {datas_invalidas_criacao} valores na coluna 'DataCriacao' não puderam ser convertidos para data.")
if datas_invalidas_entrada > 0:
    print(f"Aviso: {datas_invalidas_entrada} valores na coluna 'DataEntrada' não puderam ser convertidos para data.")


# --- 2. Criar a coluna 'Mes/Ano' ---
# Sua lógica original está perfeita. Ela cria um objeto do tipo 'Period'.
df_fechamento['Mes/Ano'] = df_fechamento['DataCriacao'].dt.to_period('M')


# --- 3. (Opcional) Criar a coluna 'Mes/Ano' como texto ---
# Se, por algum motivo, você precisar do formato 'AAAA-MM' como texto (string), 
# você pode usar o método strftime. Isso é útil para alguns tipos de gráficos ou relatórios.
# df_fechamento['Mes/Ano_Texto'] = df_fechamento['DataCriacao'].dt.strftime('%Y-%m')


# --- 4. Visualizar o resultado ---
# Mostra as colunas relevantes para confirmar que o tratamento funcionou.
colunas_para_verificar = ['DataCriacao', 'DataEntrada', 'Mes/Ano']
print("\nTipos de dados após o tratamento:")
print(df_fechamento[colunas_para_verificar].info())

print("\nVisualização das colunas tratadas:")
print(df_fechamento[colunas_para_verificar].head())



Tipos de dados após o tratamento:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147215 entries, 0 to 147214
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   DataCriacao  147215 non-null  datetime64[ns]
 1   DataEntrada  147215 non-null  datetime64[ns]
 2   Mes/Ano      147215 non-null  period[M]     
dtypes: datetime64[ns](2), period[M](1)
memory usage: 3.4 MB
None

Visualização das colunas tratadas:
              DataCriacao DataEntrada  Mes/Ano
0 2025-01-13 09:51:03.203  2025-01-13  2025-01
1 2025-01-16 11:05:49.600  2025-01-16  2025-01
2 2025-04-02 03:02:45.603  2025-03-31  2025-04
3 2025-04-02 03:02:44.907  2025-03-31  2025-04
4 2025-04-02 03:02:45.603  2025-03-31  2025-04


In [8]:
mapa_regiao_correto = {
    # --- MAPEAMENTO DA IMAGEM ORIGINAL ---
    'Gritsch Curitiba Base': 'SUL',
    'Gritsch Maringá': 'SUL',
    'Gritsch Florianópolis': 'SUL',
    'Gritsch Campo Grande': 'CENTRO OESTE',
    'Gritsch Blumenau': 'SUL',
    'Gritsch Cascavel': 'SUL',
    'Gritsch Porto Alegre': 'SUL',
    'Gritsch São Paulo (Perus)': 'SUDESTE',
    'Gritsch Goiânia': 'CENTRO OESTE',
    'Gritsch Brasília': 'CENTRO OESTE',
    'Gritsch Rondonópolis': 'CENTRO OESTE',
    'Gritsch São Paulo (Freguesia)': 'SUDESTE',
    'Gritsch Chapecó': 'SUL',
    'Gritsch Joinville': 'SUL',
    'Gritsch Londrina': 'SUL',
    'Gritsch Curitibanos': 'SUL',
    'Gritsch Criciuma': 'SUL',
    'Gritsch Sinop': 'CENTRO OESTE',
    'Gritsch Pato Branco': 'SUL',
    'Gritsch Cuiabá': 'CENTRO OESTE',
    'Gritsch Rio Verde': 'CENTRO OESTE',
    'Gritsch Laranjeiras do Sul': 'SUL',
    'Gritsch Itumbiara': 'CENTRO OESTE',
    'Gritsch Ponta Grossa': 'SUL',
    'Gritsch Curitiba ECT': 'SUL',
    'Gritsch Salvador': 'NORDESTE',
    'Gritsch Guarapuava': 'SUL',
    'GRITSCH - PMW': 'NORTE',

    # --- MAPEAMENTO DA LISTA DE NÃO ENCONTRADOS ---
    # SUDESTE
    'REFERÊNCIA SÃO PAULO': 'SUDESTE',
    'REFERÊNCIA RIBEIRAO PRETO': 'SUDESTE',
    'GRITSCH - SAO (PERUS)': 'SUDESTE',
    'GRITSCH - SAO (FREGUESIA)': 'SUDESTE',
    'GRITSCH - MATRIZ': 'SUDESTE',
    'GRITSCH - PGR': 'SUL', # 
    'GRITSCH - ITR': 'CENTRO OESTE', 

    # SUL
    'REFERÊNCIA FOZ DO IGUAÇU': 'SUL',
    'REFERÊNCIA CURITIBA': 'SUL',
    'REFERÊNCIA MARINGÁ': 'SUL',
    'GRITSCH - CWB (BASE)': 'SUL',
    'GRITSCH - FLN': 'SUL',
    'GRITSCH - BLN': 'SUL',
    'GRITSCH - POA': 'SUL',
    'GRITSCH - LDB': 'SUL',
    'GRITSCH - CSC': 'SUL',
    'GRITSCH - CWB (ECT)': 'SUL',
    'GRITSCH - CHA': 'SUL',
    'GRITSCH - CRI': 'SUL',
    'GRITSCH - PBC': 'SUL',
    'GRITSCH - CTB': 'SUL',
    'GRITSCH - GPA': 'SUL',
    'GRITSCH - MGA': 'SUL',
    'GRITSCH - CWB (DIR)': 'SUL',
    'GRITSCH - JOI': 'SUL',

    # CENTRO-OESTE
    'REFERÊNCIA GOIÂNIA LOJA': 'CENTRO OESTE',
    'REFERÊNCIA BRASILIA': 'CENTRO OESTE',
    'REFERÊNCIA CUIABÁ LOJA': 'CENTRO OESTE',
    'REFERÊNCIA SINOP': 'CENTRO OESTE',
    'GRITSCH - SNO': 'CENTRO OESTE',
    'GRITSCH - RDN': 'CENTRO OESTE',
    'GRITSCH - CBL': 'CENTRO OESTE',
    'GRITSCH - GOI': 'CENTRO OESTE',
    'GRITSCH - CGB': 'CENTRO OESTE',
    'GRITSCH - CGR': 'CENTRO OESTE',
    'GRITSCH - RVD': 'CENTRO OESTE',
    'REFERÊNCIA GOIÂNIA AV RIO VERDE': 'CENTRO OESTE',
    'GRITSCH - BSB': 'CENTRO OESTE',
    'GRITSCH - CBL': 'CENTRO OESTE',

    # NORDESTE
    'REFERÊNCIA SALVADOR': 'NORDESTE',
    'REFERÊNCIA LUIS EDUARDO MAGALHÃES': 'NORDESTE',
    'REFERÊNCIA BALSAS': 'NORDESTE',
    'GRITSCH - SSA': 'NORDESTE',

    # NORTE
    'REFERÊNCIA ARAGUAÍNA': 'NORTE',
    'REFERÊNCIA VILHENA': 'NORTE',

  

}


In [9]:
nome_coluna_filial = "FILIAL"

df_fechamento['Regiao'] = df_fechamento[nome_coluna_filial].map(mapa_regiao_correto)

filiais_nao_mapeadas = df_fechamento[df_fechamento['Regiao'].isnull()][nome_coluna_filial].unique()

if len (filiais_nao_mapeadas) >0:
    print("Filiais não mapeadas encontradas:")
    print(filiais_nao_mapeadas)
    
    df_fechamento.fillna({'Regiao': 'Regiao Nao definida'}, inplace=True)
    
else: 
    print("Todas as filiais foram mapeadas com sucesso.")

Filiais não mapeadas encontradas:
['RATEIO GRI/REF' 'RATEIO - REF' 'RATEIO - GRI' 'RATEIO - GRI/REF']


In [10]:
# Verificar se há registros com região não encontrada
regiao_nao_encontrada = df_fechamento[df_fechamento['Regiao'] == 'Regiao Nao definida']

if len(regiao_nao_encontrada) > 0:
    print(f"❌ Encontrados {len(regiao_nao_encontrada)} registros com região não definida")
    print("\nFiliais sem região:")
    print(regiao_nao_encontrada['FILIAL'].value_counts())
else:
    print("✅ Todas as regiões foram definidas corretamente!")

❌ Encontrados 28290 registros com região não definida

Filiais sem região:
FILIAL
RATEIO - REF        14098
RATEIO GRI/REF      11654
RATEIO - GRI         2232
RATEIO - GRI/REF      306
Name: count, dtype: int64


In [11]:
# Separar registros de RATEIO
df_rateio = df_fechamento[df_fechamento['Regiao'] == 'Regiao Nao definida'].copy()

# Remover RATEIOS do DataFrame principal
df_fechamento_sem_rateio = df_fechamento[df_fechamento['Regiao'] != 'Regiao Nao definida'].copy()

# Verificar a separação
print(f"DataFrame original: {len(df_fechamento):,} registros")
print(f"DataFrame RATEIO: {len(df_rateio):,} registros")
print(f"DataFrame sem RATEIO: {len(df_fechamento_sem_rateio):,} registros")
print(f"Soma: {len(df_rateio) + len(df_fechamento_sem_rateio):,} registros")

# Verificar se a separação está correta
print(f"\nFiliais no DataFrame RATEIO:")
print(df_rateio['FILIAL'].value_counts())

print(f"\nRegiões no DataFrame sem RATEIO:")
print(df_fechamento_sem_rateio['Regiao'].value_counts())

DataFrame original: 147,215 registros
DataFrame RATEIO: 28,290 registros
DataFrame sem RATEIO: 118,925 registros
Soma: 147,215 registros

Filiais no DataFrame RATEIO:
FILIAL
RATEIO - REF        14098
RATEIO GRI/REF      11654
RATEIO - GRI         2232
RATEIO - GRI/REF      306
Name: count, dtype: int64

Regiões no DataFrame sem RATEIO:
Regiao
CENTRO OESTE    38832
SUDESTE         35333
SUL             23777
NORDESTE        11807
NORTE            9176
Name: count, dtype: int64


In [13]:
df_fechamento_sem_rateio['Natureza_Correta'] = df_fechamento_sem_rateio['Natureza_Correta'].where(
    df_fechamento_sem_rateio['Natureza_Correta'].notna(), 
    df_fechamento_sem_rateio['NaturezaFinanceira']
)

In [16]:
# PADRONIZAÇÃO DAS NATUREZAS
# Sempre priorizar as que têm código na frente

import pandas as pd
import re

def criar_mapa_padronizacao_naturezas():
    """
    Cria o mapeamento para padronizar as naturezas
    Prioriza sempre as versões com código
    """
    
    # Mapeamento baseado na sua análise
    mapa_padronizacao = {
        # LAVAGEM - padronizar para a versão com código
        'LAVagem': '03.01 - LAVAGEM',  # corrigir capitalização também
        'LAVAGEM': '03.01 - LAVAGEM',
        '03.01 - LAVagem': '03.01 - LAVAGEM',
        
        # MANUTENÇÃO DE VEÍCULOS - padronizar para a versão com código
        'MANUTENÇÃO DE VEÍCULOS': '03.03 - MANUTENÇÃO DE VEÍCULOS',
        
        # DEDETIZAÇÃO - padronizar para a versão com código
        'DEDETIZAÇÃO DE VEÍCULO': '07.07 - DEDETIZAÇÃO DE VEÍCULO',
        
        # COMBUSTÍVEL - padronizar para a versão com código
        'COMBUSTÍVEL': '02.01 - COMBUSTÍVEL',
        
        # IPVA ANUAL - padronizar para a versão com código
        'IPVA (ANUAL)': '07.02 - IPVA (ANUAL)',
        
        # ARLA - padronizar para a versão com código
        'ARLA': '02.02 - ARLA',
        
        # TAXI - padronizar para a versão com código
        'TAXI': '16.01 - TAXI',
        
        # Naturezas que não têm versão com código - manter como estão
        'ESTACIONAMENTO': 'ESTACIONAMENTO',
        'MULTAS DE TRÂNSITO': 'MULTAS DE TRÂNSITO',
        'LICENCIAMENTO': 'LICENCIAMENTO',
        'CUSTAS': 'CUSTAS',
        'SUBCONTRATAÇÃO DE TRANSPORTE': 'SUBCONTRATAÇÃO DE TRANSPORTE',
        'SEGURO DE VEÍCULOS (FACULTATIVO)': 'SEGURO DE VEÍCULOS (FACULTATIVO)',
        'PEDÁGIO': 'PEDÁGIO'
    }
    
    return mapa_padronizacao

def padronizar_naturezas(df):
    """
    Padroniza as naturezas no DataFrame
    """
    df_padronizado = df.copy()
    
    # Criar mapa de padronização
    mapa_padronizacao = criar_mapa_padronizacao_naturezas()
    
    print("=== PADRONIZAÇÃO DAS NATUREZAS ===\n")
    
    # Aplicar padronização na Natureza_Correta
    print("Padronizações que serão aplicadas:")
    alteracoes_feitas = 0
    
    for natureza_antiga, natureza_nova in mapa_padronizacao.items():
        # Contar quantos registros serão alterados
        count_antes = (df_padronizado['Natureza_Correta'] == natureza_antiga).sum()
        
        if count_antes > 0:
            print(f"  '{natureza_antiga}' → '{natureza_nova}' ({count_antes:,} registros)")
            
            # Aplicar a substituição
            df_padronizado.loc[df_padronizado['Natureza_Correta'] == natureza_antiga, 'Natureza_Correta'] = natureza_nova
            alteracoes_feitas += count_antes
    
    print(f"\nTotal de registros alterados: {alteracoes_feitas:,}")
    
    return df_padronizado

def verificar_padronizacao(df_antes, df_depois):
    """
    Verifica o resultado da padronização
    """
    print("\n=== VERIFICAÇÃO DA PADRONIZAÇÃO ===\n")
    
    print("ANTES da padronização:")
    naturezas_antes = df_antes['Natureza_Correta'].value_counts()
    print(f"  Total de naturezas únicas: {len(naturezas_antes)}")
    
    print("\nDEPOIS da padronização:")
    naturezas_depois = df_depois['Natureza_Correta'].value_counts()
    print(f"  Total de naturezas únicas: {len(naturezas_depois)}")
    
    print(f"\nRedução de naturezas únicas: {len(naturezas_antes) - len(naturezas_depois)}")
    
    print("\nNaturezas finais (ordenadas por quantidade):")
    for i, (natureza, count) in enumerate(naturezas_depois.items(), 1):
        print(f"  {i:2d}. '{natureza}' → {count:,} registros")
    
    return naturezas_depois

# ===== EXECUÇÃO =====

# 1. Aplicar padronização
df_fechamento_padronizado = padronizar_naturezas(df_fechamento_sem_rateio)

# 2. Verificar resultado
naturezas_finais = verificar_padronizacao(df_fechamento_sem_rateio, df_fechamento_padronizado)

# 3. Atualizar a variável principal (substitua df_fechamento_sem_rateio pela versão padronizada)
df_fechamento_sem_rateio = df_fechamento_padronizado.copy()

print("\n✅ Padronização concluída!")
print("✅ DataFrame atualizado: df_fechamento_sem_rateio")

# 4. Verificação final - mostrar apenas as top 10
print(f"\n=== TOP 10 NATUREZAS FINAIS ===")
top_10_naturezas = naturezas_finais.head(10)
for i, (natureza, count) in enumerate(top_10_naturezas.items(), 1):
    perc = (count / len(df_fechamento_sem_rateio)) * 100
    print(f"{i:2d}. '{natureza}' → {count:,} registros ({perc:.1f}%)")

# 5. Verificar se ainda há duplicações
print(f"\n=== VERIFICAÇÃO DE POSSÍVEIS DUPLICAÇÕES RESTANTES ===")
naturezas_sem_codigo = []
naturezas_com_codigo = []

for natureza in naturezas_finais.index:
    if re.match(r'^\d{2}\.\d{2} - ', str(natureza)):
        naturezas_com_codigo.append(natureza)
    else:
        naturezas_sem_codigo.append(natureza)

print(f"Naturezas com código: {len(naturezas_com_codigo)}")
print(f"Naturezas sem código: {len(naturezas_sem_codigo)}")

if naturezas_sem_codigo:
    print("\nNaturezas sem código (verificar se há similares com código):")
    for nat in naturezas_sem_codigo:
        count = naturezas_finais[nat]
        print(f"  - '{nat}' → {count:,} registros")

=== PADRONIZAÇÃO DAS NATUREZAS ===

Padronizações que serão aplicadas:
  '03.01 - LAVagem' → '03.01 - LAVAGEM' (839 registros)
  'ESTACIONAMENTO' → 'ESTACIONAMENTO' (4 registros)
  'MULTAS DE TRÂNSITO' → 'MULTAS DE TRÂNSITO' (3 registros)
  'LICENCIAMENTO' → 'LICENCIAMENTO' (3 registros)
  'CUSTAS' → 'CUSTAS' (1 registros)
  'SUBCONTRATAÇÃO DE TRANSPORTE' → 'SUBCONTRATAÇÃO DE TRANSPORTE' (1 registros)
  'SEGURO DE VEÍCULOS (FACULTATIVO)' → 'SEGURO DE VEÍCULOS (FACULTATIVO)' (1 registros)
  'PEDÁGIO' → 'PEDÁGIO' (1 registros)

Total de registros alterados: 853

=== VERIFICAÇÃO DA PADRONIZAÇÃO ===

ANTES da padronização:
  Total de naturezas únicas: 24

DEPOIS da padronização:
  Total de naturezas únicas: 23

Redução de naturezas únicas: 1

Naturezas finais (ordenadas por quantidade):
   1. '03.03 - MANUTENÇÃO DE VEÍCULOS' → 96,026 registros
   2. '04.01 - RASTREAMENTO E MONITORAMENTO DE VEÍCULO' → 12,559 registros
   3. '03.01 - LAVAGEM' → 3,708 registros
   4. '03.04 - ACESSÓRIOS DE VE

In [21]:
df_rateio.drop(columns=['Regiao'], inplace=True)

In [24]:
df_fechamento_sem_rateio.info()

<class 'pandas.core.frame.DataFrame'>
Index: 118925 entries, 1 to 147214
Data columns (total 42 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   IdNF                          118925 non-null  int64         
 1   NumeroNF                      118925 non-null  object        
 2   OrdemServico                  118925 non-null  object        
 3   Ocorrencia                    118925 non-null  object        
 4   OrdemCompra                   118925 non-null  object        
 5   Placa                         118925 non-null  object        
 6   IdVeiculo                     118925 non-null  int64         
 7   DescricaoItem                 118925 non-null  object        
 8   TipoItem                      118925 non-null  object        
 9   Tipo                          118925 non-null  object        
 10  TipoOrdemCompra               118925 non-null  object        
 11  IdGrupoDespesa    